#### Урок 2. Парсинг данных. HTML, DOM, XPath
Написать приложение и функцию, которые собирают основные новости с сайта на выбор dzen.ru, lenta.ru, mail.ru . Для парсинга использовать XPath
Структура данных должна содержать:
* название источника
* наименование новости
* ссылку на новость
* дата публикации

минимум один сайт максимум все

Задание считается выполненным, если:
- код задокументирован, парсер работает корректно
- репозиторий в github имеет readme.md с описанием и структурой проекта (если сдача github, gitlab )
- есть скриншоты результата выполнения задания

In [2]:
from pprint import pprint
import requests
from lxml import html
import datetime

In [3]:
my_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}

In [4]:
def requests_mail_news():
    url = 'https://news.mail.ru/'
    response = requests.get(f'{url}', headers=my_headers)
    root = html.fromstring(response.text)
    news_block= root.xpath("//div[contains(@class,'daynews__')]/a | //ul[@data-module='TrackBlocks']/li[@class='list__item']/a")
    base_news = []

    for i, item in enumerate(news_block, 1):
        news = {}
        if i < 6:
            name_news = item.xpath(".//span[contains(@class, 'photo__title')]/text()")
        else:
            name_news = item.xpath(".//text()")
        link_news = item.xpath(".//@href")
        response_2 = requests.get(link_news[0], headers=my_headers)
        dom = html.fromstring(response_2.text)
        name_source = dom.xpath("//span[contains(@class, 'breadcrumbs__item')]//a//text()")
        date_news_info = dom.xpath("//span[contains(@class, 'breadcrumbs__item')]//span[contains(@class, 'note__text')]/@datetime")
        date_news = date_news_info[0][:10].split('-')
        date_news = datetime.date(int(date_news[0]), int(date_news[1]), int(date_news[2])).strftime('%d.%m.%Y')
        news['name_source'] = name_source[0]
        news['name_news'] = name_news[0].replace(u'\xa0', ' ')
        news['link_news'] = link_news[0]
        news['date_news'] = date_news
        base_news.append(news)

    return base_news

In [8]:
mail_news = requests_mail_news()